<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/twyncoder/tf-hands-on/blob/master/L06_TransferLearning_clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

<center><a href="https://centroia.uva.es/"> <img src="logo-UVAIA-original.png" alt="Header" style="width: 300px;"/> </a></center>

# Redes de Aprendizaje Profundo básicas con Keras y Tensorflow.
## *Transfer Learning*

## 0. Preparación del entorno 

In [ ]:
# Common imports
import os
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow import keras

# Confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_name + "." + fig_extension)
    print("Saving figure", fig_name)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

def print_history(history,title=None, extension='png'):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    #plt.gca().set_ylim(0, 1)
    plt.xlabel("epochs")
    if(title!=None):
        plt.title(title)
        save_fig(title,fig_extension=extension)

### Información de versiones

In [ ]:
tf.__version__

### Comprobar si disponemos de una GPU

In [ ]:
tf.config.list_physical_devices('GPU')

## 1. Inspeccionar los datos y crear subconjuntos train, test, validation

In [ ]:
!mkdir cracktyres
!unzip cracktyres.zip

In [ ]:
# Función para cargar las imágenes y etiquetas desde los archivos CSV
def load_data(csv_file, img_dir):
    data = pd.read_csv(csv_file)
    file_paths = data.iloc[:, 0].values
    labels = data.iloc[:, 1:].values

    paths = [os.path.join(img_dir, file) for file in file_paths]

    return paths, labels

# Función para cargar y preprocesar las imágenes
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalización a [0, 1]
    return image, label

# Función para crear un dataset tf.data a partir de las rutas y etiquetas
def create_dataset(paths, labels, batch_size):
    path_ds = tf.data.Dataset.from_tensor_slices(paths)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    image_label_ds = tf.data.Dataset.zip((path_ds, label_ds))

    ds = image_label_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [ ]:
# Cargar las rutas y etiquetas de las imágenes
train_paths, train_labels = load_data('./cracktyres/train/_classes.csv', './cracktyres/train')
valid_paths, valid_labels = load_data('./cracktyres/valid/_classes.csv', './cracktyres/valid')
test_paths, test_labels = load_data('./cracktyres/test/_classes.csv', './cracktyres/test')

# Crear los datasets
batch_size = 16
train_dataset = create_dataset(train_paths, train_labels, batch_size)
valid_dataset = create_dataset(valid_paths, valid_labels, batch_size)
test_dataset = create_dataset(test_paths, test_labels, batch_size)

In [ ]:
# Mostrar las imágenes del conjunto de entrenamiento
def show_images(dataset, num_images):
    plt.figure(figsize=(15, 15))
    for images, labels in dataset.take(1):  # Tomar el primer batch
        for i in range(num_images):
            ax = plt.subplot(4, 8, i + 1)
            plt.imshow(images[i])
            label = 'Problema' if labels[i][0] == 1 else 'OK'
            plt.title(label)
            plt.axis('off')
    plt.show()



In [ ]:
# Imprimir las primeras 32 imágenes del dataset de entrenamiento
show_images(train_dataset, batch_size)

## 2. Entrenar red neuronal para clasificación binaria de neumáticos

In [ ]:
# Ahora puedes usar estos datasets para entrenar tu modelo
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # 2 para clasificación binaria con one-hot
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=25)

In [ ]:
print_history(history,"L06_cnn_basic")

**¡AHORA TÚ!** 
- ¿Qué tal ha ido el aprendizaje? ¿Puede estar habiendo _overfitting_? Si es así, ¿a qué crees que es debido?

In [ ]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")

## 3. Utilizar Data augmentation para prevenir el _overfitting_

In [ ]:
def preprocess_image(image_path, label, augment=False):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalización a [0, 1]

    if augment:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_brightness(image, max_delta=0.1)
        image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
        image = tf.image.random_saturation(image, lower=0.9, upper=1.1)
        image = tf.image.random_hue(image, max_delta=0.1)

    return image, label


# Función para crear un dataset tf.data a partir de las rutas y etiquetas
def create_dataset(paths, labels, batch_size, augment=False):
    path_ds = tf.data.Dataset.from_tensor_slices(paths)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    image_label_ds = tf.data.Dataset.zip((path_ds, label_ds))

    ds = image_label_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds


Podemos probar a entrenar el mismo modelo para ver el efecto de _Data Augmentation_

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # 2 para clasificación binaria con one-hot
])

In [ ]:
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history2 = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=25)

In [ ]:
print_history(history2,"L06_cnn_augment")

In [ ]:
model.evaluate(test_dataset)

... Y también podemos entrenar nuevos modelos

**¡AHORA TÚ! (OPCIONAL)** 
- Prueba a entrenar modelos más avanzados y utilizando distintas técnicas de regularización.
- Piensa en utilizar valores de _learning rate_ pequeños o, mejor aún... técnicas dinámicas de ajuste.

In [ ]:
# TODO

# 4. Reconocimiento con redes pre-entrenadas

- Cuando disponemos de pocos datos para los entrenamientos, es buena idea aprovechar modelos ya pre-entrenados que puedan sernos útiles.
- Como vimos anteriormente, Keras dispone de algunos modelos ya pre-entrenados con el dataset de _imagenet_ que son muy útiles para clasificación.
- Ahora aprovecharemos la extracción de características aprendida por uno de estos modelos para crear nuestro clasificador de neumáticos.

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [ ]:
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

Para adaptar la red es necesario:
- No alterar (inicialmente) los valores entrenados de los kernels de convolución de las primeras capas.  
- Ajustar la capa _fully connected_ o `Dense()` de salida al número de clases que necesitemos.

In [ ]:
# Freeze base model
base_model.trainable = False

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(2, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model7 = keras.Model(inputs,outputs)

In [ ]:
model7.summary()

Vamos a prevenir el _overfitting_ empleando un callback de _Early Stopping_

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,verbose=1)

In [ ]:
model7.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
history7 = model7.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=100,
                    callbacks=[early_stopping_cb,model_checkpoint_cb])

In [ ]:
print_history(history7,"L06_PretrainedVGG")

In [ ]:
model7.evaluate(test_dataset)

# 5. Fine tunning

Los resultados anteriores no están nada mal, ¿Podemos mejorarlo?
- Vamos a reentrenar el modelo completo permitiendo ahora que se modifiquen los kernels de las capas de extracción de características.
- Utilizaremos un valor de _learning rate_ muy pequeño para no perjudicar e entrenamiento anterior.


In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model7.compile(#optimizer=keras.optimizers.RMSprop(learning_rate = 1e-05),
              optimizer=keras.optimizers.Adam(learning_rate=1e-04, beta_1=0.9, beta_2=0.999),
              loss = 'categorical_crossentropy' , metrics = 'accuracy')

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,verbose=1)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("model_7tuned.keras", save_best_only=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=3,verbose=1)


In [ ]:
history73 = model7.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=15,
                    callbacks=[early_stopping_cb,model_checkpoint_cb,lr_scheduler])

In [ ]:
print_history(history73,"L07_VGG_finetuning_callbacks")

In [ ]:
model7.evaluate(test_dataset)